# Class Activation Map

## 개요

* XAI(Explainable Artificial Intelligence, 설명 가능한 인공지능) : 모델과 추론의 신뢰성에 대한 답을 찾는 분야

### 실습 목표

1. 분류 모델의 활성화 맵을 이해합니다.
2. 다양한 활성화 맵을 구하는 방법을 알아갑니다.
3. 약지도학습(weakly supervised learning)을 이해합니다.

### 학습 내용

1. Explainable AI
2. CAM: Class Activation Map
3. Grad-CAM
4. ACoL: Adversarial Complementary Learning
5. 생각해 보기

## Explainable AI

* 모델이 잘못된 답변을 준다면 어떻게 개선할 수 있는지?
* 모델이 잘 작동한다면 왜 이런 선택을 했는지?

### Image Classification

* local feature를 추출하기 위해 CNN으로 구성된 특성 추출용 backbone network가 있음.
* backbone에서 추출된 feature map을 fully connected layer에 통과시켜 얻은 logit
* logit을 softmax activation function에 통과시켜 class에 속할 확률으 구함.

* [관련설명](https://opentutorials.org/module/3653/22995)
* logit 과 sigmoid는 서로 역함수.
* 2개의 class를 대상으로 정의하던 logit을 K개의 class를 대상으로 일반화하면 softmax 함수가 유도됨.
* softmax 함수에서 K를 2로 놓으면 sigmoid 함수로 환원.
* sigmoid 함수를 K개의 클래스를 대상으로 일반화시 softmax 함수가 유도.

$$ t = logit(y) = ln(\frac{y}{1-y}) $$
$$ y = sigmoid(t) = \frac{1}{1+exp(t)} $$

## CAM: Class Activation Map

* CAM : Class Activation Map
* 모델이 어떤 곳을 보고 어떤 클래스임을 짐작하고 있는지 확인할 수 있는 지도
* CAM을 얻기 위해 GAP를 사용
* 이를 통해 어떤 class가 어느 영역에 의해서 활성화 되었는지 알수 있음

### GAP(Global Average Pooling)

### MaxPooling

<img src = 'image/MaxPooling.png'>

* 2x2로 나누어서 최댓값을 찾아서 넣습니다.
* 2x2 영역과 2만큼의 stride (f = 2, s = 2)
* f, s가 MaxPooling의 hyperparameter가 된다.
* 쓰게된 이유 : 성능과 직관이 좋음 + 정확인 이유는 없다.
* 경사하강을 할 수 없다.

<img src = 'image/MaxPooling_2.png'>

* input 5x5에서 hyperparameter : f = 3, s = 1로 보자.
* 공식 $\frac{n + 2p - f}{s} +1$ 으로 3x3이 나온다.
* 2차원이 아닌 3차원이라면 채널에 따라 MaxPooling을 진행하게 된다.
* $ 5\times5\times n_{c} \rightarrow 3\times3\times n_{c}$ ($n_{c}$ = channel 갯수)
* MaxPooling은 각 채널에 개별적으로 적용

### Average Pooling

<img src = 'image/AveragePooling.png'>

* 2x2로 나누어서 평균을 넣습니다.
* 2x2 영역과 2만큼의 stride (f = 2, s = 2)
* f, s가 AveragePooling의 hyperparameter가 된다.
* MaxPooling을 AveragePooling보다 더 많이 사용합니다.
* Pooling에서는 역전파가 가능한 변수가 없습니다.
* 직접 또는 교차 검증을 통해 정해진 Hyperparameter이기 때문.

### Global Average Pooling

* Network in Network 논문에서는 fully connected layer 대신 GAP 연산을 한다.
* GAP(Global Average Pooling)은 매 채널별로, average pooling을 채널 값 전체에 global하게 적용합니다.
* 채널마다 딱 한개의 숫자로 요약되어 1x1크기의 채널 개수만큼의 vector가 생성됩니다.(dimension reduction)
* 결국 average pooling임으로 역전파가 가능한 변수가 없습니다.
* 그렇다고 backpropagation이 안되는건 아닙니다.
* backpropagation의 연산과 pooling의 변수없음은 개별적으로 생각해야합니다.
* 어떻게 보면 pooling에서 그냥 연산되는 값들이 그냥 넘어간다고 생각하면 됩니다
* [backpropagation and pooling](https://ratsgo.github.io/deep%20learning/2017/04/05/CNNbackprop/)

### CAM

* 설명 가능 AI 기법
* Class Activation Map : 클래스가 활성화되는 지도.
* CNN : Convolution layer들과 Flatten layer를 통해 Feature map을 뽑아내고(Feature Extraction)\
Fully Connected layer들과 softmax 함수를 통해 Classification을 수행.
* CNN은 kernel window(filter)에 따라 특성을 추출하므로 CNN layer를 적용한 특성맵에도 위치정보가 남습니다.
* 해당 특성맵에 GAP를 적용
* 그리고 softmax를 가진 dense layer(bias 없음)를 적용

* 각 Class에 대한 정보들이 여러 채널에 걸쳐 나타남.
* GAP를 통해 각 채널(feature map)에 대한 중요도를 요약하고 이를 softmax로 개별 채널의 중요도를 결정.
* Class 별로 softmax layer를 통해 각 채널의 가중합을 구하면 각 클래스의 활성화 맵의 어떤 부분을 주로 활성화시키는지 확인.
* 특성 맵은 기존의 CNN의 출력값 크기와 같기 때문에 interpolation을 통해 원본 이미지 크기로 확대해준다.

* k개의 채널이 있다고 가정.
* k = 1, 2, ..., n 인 k번째 채널에서 $w_{k}^{c}$는 각 클래스 노드(c)노드와 k번째 채널 사이의 가중치 값.
* $f_{k}{(x,y)}$는 k번째 채널의 x, y 요소의 활성화 값(활성화 함수를 통해 나온 값)
* 이 두가지를 곱하고 모든 채널과 x, y 축에 대해 더해줌으써 클래스별 점수 $S_{c}$를 구할 수 있습니다.
* x, y는 feature에서의 위치 값(픽셀 위치, input의 좌표가 아님)

$$ S_{c} = \sum_{k}w_{k}^{c} \sum_{x,y}f_{k}(x,y) = \sum_{x,y}\sum_{k} w_{k}^{c} f_{k}(x,y). $$

* CAM을 얻기 위해서는 위처럼 클래스별 점수르 얻기 위해  모든 위치의 활성화 정도를 더해준 것과 달리
* 각 x,y 위치에서 k개의 채널만 더해주어 위치정보가 남도록 합니다.
* 이를 통해 얻어진 CAM은 채널별 활성화 정도가 가중합인 $M_{c}(x,y)$가 됩니다.
* $M_{c}(x,y)$은 모델이 클래스 c에 대해 각 위치를 얼마나 잘보고 있는지 나타냅니다.

$$ M_{c}(x,y) = \sum_{k}w_{k}^{c} f_{k}(x,y). $$

* CAM에서 마지막 layer에서 원인 분석이 가능한 이유?
  - 마지막 convlution feature map이 가진 정보량이 많음
  - 마지막 feature map 내 1개 값은 원본 이미지에서 많은 부분을 요약한 결과

## Grad-CAM

### 기본 구조

* 기존의 CAM은 GAP를 통해 $f_{k}$를 구하고, 그 뒤로 fully connected layer $w_{k}^{c}$를 추가로 붙여야 합니다.
* 또한, 가장 마지막 CNN layer의 결과물만을 시각화 할 수 있었습니다.
* CAM의 한계점
  1. Global Average Pooling을 반드시 사용해야함.
  2. 뒷부분에 또 다시 fine tuning을 해야함
  3. 마지막 Convolution layer에 대해서만 CAM 추출 가능
* Grad-CAM의 이러한 CAM의 한계를 개선하고 다양한 모델의 구조를 해석 할 수 있는 방법을 제안.

* Grad = Gradient-weighted
* GAP 사용안함
* CNN 기반의 네트워크는 굳이 모델 구조를 변경할 필요가 없음
* Gradient는 특정 class에 특정 input이 주는 영향력
* 분류 문제가 아닌 다른 문제에서도 사용가능합니다.
* 밑은 Grad-CAM의 구조입니다.

<img src = 'image/Grad_CAM.png' height = 70% width = 70%>  
  
출처 : Grad-CAM: Visual Explanations from Deep Networks via Gradient-based Localization paper

* 이미지를 입력하면 CNN을 거쳐서 Feature Maps를 구합니다.
* 그 뒤로 다양한 문제에 따라 layer를 거치게 됩니다.
* Image Classification : 이미지 분류
* Image Captioning : 이미지에 대한 설명을 만들어냄
* Visual Question Answering(VQA) : 어떤 질문과 이미지가 주어졌을 때 이에 대한 답변을 내는 업무입니다.

### Gradient을 통한 Weight Scroe 계산

* CAM에서는 fully connected layer의 가중치를 통해서 어떤 클래스에 대한 각 채널의 중요도 또는 가중치를 알아냈습니다.
* Grad-CAM는 gradient를 사용합니다.
* 원하는 class에 대해서 관찰하는 layer로 들어오는 gradient를 구할 수 있다면
* 해당 클래스를 활성화할 때, 레이어의 특성맵에서 어떤 채널이 중요하게 작용하는지 알 수 있습니다.
* 아래 식은 클래스에 대해 backpropagation을 통해 얻은 k번째 채널의 gradient를 사용해 가중치 점수(Weight Score)를 구하는 식입니다.

$$ a_{k}^{c} = \overbrace{\frac{1}{Z}\sum_{i}\sum_{j}}^{global \, average \, pooling} \ 
 \underbrace{\frac {\partial y^{c}} {\partial A_{ij}^{k}}}_{gradient \, via \, backprop} $$

* $y$ : 출력값, $A$ : 활성화 맵, i,j : x,y축
* Z : 전체 맵 크기(i = 1, 2, ..., u, j = 1, 2, ..., v 라면 Z = u * v)
* $y^{c}$는 class c에 대한 출력값
* $A_{ij}^{k}$ : feature map 원소
* 출력값을 각각의 feature map 원소로 미분 : gradient 값이 나옴, 이것의 평균을 구함
* Feature Map의 각 원소가 특정 class에 주는 영향력 : Gradient
* Gradient을 통해 feature map의 가중치를 계산
* Weight를 기존의 CAM에서는 학습을 통해 구했는데 Grad-CAM에서는 gradient 값을 통해 구함
* 이 정보들을 합쳐 활성화 맵에서 어떤 클래스의 위치에 따른 활성화를 보기 위해
* k번째 활성화 맵과 이 가중치를 곱해주어 ReLU 활성화 함수를 통해서 클래스에 따른 Grad-CAM을 얻습니다.
* 이 수식을 시각화한 것이 맨 위에서 소개한 고양이와 개에 대한 Grad-CAM의 (c),(i)번째 예시입니다.

$$ L_{Grad-CAM}^{c} = ReLU \underbrace {(\sum_{k}a_{k}^{c}A^{k})}_{linear \, combination} $$

## ACoL : Adversarial Complementary Learning

* CAM은 클래스와 이미지만을 데이터로 학습하여 위치정보까지 얻을 수 있음.
* 약지도학습(weakly supervised learning) : 직접적인 정답 위치 정보를 주지 않고 간접적인 정보를 활용하여 원하는 정보를 얻어낼 수 있도록 모델을 학습하는 방식
* CAM, Grad-CAM, ACoL은 약지도학습 기법을 활용해 물체 검출(object detection)을 수행할 수 있음.

### 약지도학습(weakly supervised learning)

* 약지도학습과 준지도학습(semi-supervised learning)

* incomplete supervision : 학습 데이터 중 일부에만 라벨이 달린 경우. 일반적으로 말하는 준지도학습과 같은 경우임.
  * (ex: 개와 고양이 분류 학습 시 10,000개의 이미지 중 1,000개만 라벨이 있는 경우)
* inexact supervision : 학습데이터의 라벨이 충분히 정확하게 달려있지 않은 경우.
  * (ex: 개나 고양이를 Object Detection 또는 Semantic Segmentation해야 하지만  
  이미지 내에 정확한 bounding box는 주어져 있지 않고 이미지가 개인지 고양인지 정보만 라벨로 달려있는 경우)
* inaccurate supervision : 학습 데이터에 Noise가 있는 경우
  * (ex: 개나 고양이의 라벨이 잘못 달린 경우)
* 약지도학습이란 위 3가지 경우를 포괄적으로 일컫도록 사용되곤 하지만, 요번에 다룰 것은 inexact supervison 입니다.

* Grad-CAM에서 class의 특징을 두드러지게 하는 영역의 가중치 점수를 구할 수 있습니다.
* 해당 가중치 점수를 제거하게 되면 Image classification 모델에서 해당 클래스에 대한 prediction이 바뀌게 될 수 있습니다.
* 가중치 점수를 제거 했을 때 prediction이 바뀌도록 하는 가중치 영역을 모으면 bounding box 라벨을 보지않고도 object detection이 가능합니다.
* 위와 비슷한 개념을 통해 semantic segmentation 또한 할 수 있습니다.

### Adversarial Complementary Learning

<img src = 'image/ACoL.png' height = 70% width = 70%>  
  
출처 : Adversarial Complementary Learning for Weakly Supervised Object Localization paper

* 위 그림은 ACoL의 구조입니다.
* Adversarial 은 GAN에서 나온 것처럼 적대적이다라는 뜻입니다.
* 보시다시피 2가지 갈래로 나눕니다.
* CAM을 만들기 위해 활용한 특성 맵을 두 가지로 분류한 것 입니다.

* CAM과 Grad-CAM은 Weakly-supervised 방식의 Object Detection, Segmentation의 가능성을 보여줬습니다.
* 이들 방식을 통해 나온 활성화 맵을 보면 가장자리보다는 특징이 주로 나타내는 위치에 중점적으로 활성화되는 모습을 볼 수 있습니다.
* 하지만 Object Detection은 부분적 특성이 아닌 전체적인 형태와 윤곽을 정확하게 구분해 내는 것이 중요합니다.
* 이것을 위해 CAM 모델이 특정 부위에 집중해 학습하는 것을 막기 위해 ACoL은 브랜치를 2가지로 두어 너무 높은 점수를 지워줌으로써 주변 특성 또한 반영하게 함.
* 이러한 과정을 논문에서는 Adversial(적대적) 학습방법이라 합니다.

* 브랜치들을 살펴봅시다.
* 먼저 위쪽의 주황색 브랜치를 보면 GAP를 거쳐 softmax layer인 classifier A를 거치게 됩니다.
* 해당 브랜치는 loss로 학습됩니다.

* ACoL은 여기서 얻어진 활성화 맵을 적대적인 방법으로 사용합니다.
* 일정 값 이상 활성화된 활성화 맵을 지우도록 하는 것입니다.
* Classiier A는 쉽게 전체적인 이미지를 보고 클래스를 판별할 수 있는
* 반면에 Classifier B는 A의 CAM에서 크게 활성화된 영역을 지운 활성화 맵에서 분류를 해야 하기 때문에 더 어려운 문제를 푸는 것으로 볼 수 있습니다(?)

* 이렇게 두 가지 Classifier A와 Classifier B를 학습시킴으로써 더 넓은 영역을 판별의 근거로 삼을 수 있습니다.
* 이러한 과정들로 모델은 쉽게 맞출 수 있는 샘플을 어렵게 다시 학습하는 Adversarial Complementary Learning 과정을 거치게 됩니다.
* 이를 통해 ACoL은 기존 CAM의 활성화 영역보단 물체의 전반적인 영역을 CAM이 활성화하는 효과가 나옵니다.

### 1x1 Conv

* CAM에서 대상이 네트워크에 feed forward하고 활성화 맵과 가중치 계산을 다시 해주어야 합니다.
* 이 과정에서 관찰하고자 하느 분류모델 feed forward와 별개의 작업이므로 물체 검출을 위한 모델로 사용하기 위해서는 별도의 연산이 필요합니다.

* ACoL은 이를 개선하기 위해 kernel size 1x1, 출력 채널의 개수는 분류하고자하는 class 개수를 가진 convolution layer를 feature map에 사용하고
* 여기에 GAP를 적용하여 사용합니다.
* Convolution layer의 출력 값은 곧바로 활성화 맵이 됩니다.
* 밑의 그림의 Convolution layer를 통해 구한 활성화 맵과 CAM을 통해 구한 활성화 맵의 비교입니다.

<img src = 'image/Grad_CAM_2.png' height = 60% width = 60%>  
  
출처 : Adversarial Complementary Learning for Weakly Supervised Object Localization paper